In [4]:
d = '../data/pre/' # raw data directory
print('dev stat 22666633')
import os
import pandas as pd
import numpy as np
from time import time
from sklearn.metrics import log_loss

from keras.models import load_model, Sequential, Model
from keras.utils import plot_model, to_categorical
from keras.callbacks import TensorBoard, ModelCheckpoint, Callback
from keras.layers import Dense, Embedding, LSTM, GRU, SimpleRNN, BatchNormalization
from keras.layers import Dropout, Bidirectional, Flatten, Input, Reshape
from keras.layers.merge import Concatenate, Add, concatenate, add

ac = pd.read_csv('%sapp_categories.csv' % d)
ui = pd.read_csv('%suser_installedapps.csv' % d)
ua = pd.read_csv('%suser_app_actions.csv' % d)

tr_ori = pd.read_csv('../data/pre/new_generated_train.csv', index_col=0)
te_ori = pd.read_csv('../data/pre/new_generated_test.csv', index_col=0)
# features = [ 'positionType', 'connectionType', 'age', 'haveBaby', 'telecomsOperator',
#             'gender', 'education', 'clickTime_h', 'clickTime_d', 'weekDay',
#             'marriageStatus', 'appPlatform', 'clickTime_m']

dev stat 22666633


/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
va = tr_ori.sample(frac=.1, random_state=3)
tr = tr_ori.drop(va.index, axis=0)

va_y = va.label.values.reshape(-1,1)
tr_y = tr.label.values.reshape(-1,1)
va_x = va[f].values.reshape(-1,1)
tr_x = tr[f].values.reshape(-1,1)

va_ = va.groupby(f).apply(lambda df: np.mean(df.label))
tr_ = tr.groupby(f).apply(lambda df: np.mean(df.label))


tr_stat = tr.groupby(f).apply(lambda df: len(df))

print(tr_stat.values)

f = 'appCategory'

total_tr = 2*max(tr_stat.values)
new_x = []
new_y = []
for cate,occ in zip(tr_stat.index, tr_stat.values):
    tmp = tr.loc[tr[f]==cate]
    x = tmp[f].values
    y = tmp.label.values
    x = list(x)
    y = list(y)
    x = x*(int(total_tr/len(x)))
    y = y*(int(total_tr/len(y))) 
    print(len(x))   
    new_x += x
    new_y += y
new_x = np.array(new_x).reshape(-1,1)
new_y = np.array(new_y).reshape(-1,1)
print('Length of new x:', len(new_x))

[    133  430709   26285  386918     205  190203  282884   13789 1725149
   40967  192925   30002      59   54347]
3450286
3445672
3443335
3095344
3450150
3423654
3394608
3447250
3450298
3441228
3279725
3450230
3450261
3423861
Length of new x: 47645902


In [6]:
# no ajust
np.random.seed(323)
i = Input(shape=(1,))
o = Embedding(np.max(tr_x)+1, 64)(i)
o = Flatten()(o)
o = Dense(64, activation='tanh')(o)
o = Dense(1, activation='sigmoid')(o)
model_ = Model(i,o)
model_.summary()
model_.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['binary_crossentropy'])
model_.fit(tr_x, tr_y,validation_data=(va_x,va_y), verbose=1, epochs=2, batch_size=1024,  shuffle=True)

print('\nUnbalanced model predict:\n', model_.predict(va_.index))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1, 64)             32256     
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 36,481
Trainable params: 36,481
Non-trainable params: 0
_________________________________________________________________
Train on 3374575 samples, validate on 374953 samples
Epoch 1/2
3374575/3374575 [==============================] - 8s - loss: 0.1152 - bina

In [7]:
# balanced
np.random.seed(323)
i = Input(shape=(1,))
o = Embedding(np.max(new_x)+1, 64)(i)
o = Flatten()(o)
o = Dense(64, activation='tanh')(o)
o = Dense(1, activation='sigmoid')(o)
model = Model(i,o)
model.summary()
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['binary_crossentropy'])
model.fit(new_x, new_y,validation_data=(va_x,va_y), verbose=1, epochs=2, batch_size=10240,  shuffle=True)

print('\nBalanced model predict:\n', model.predict(va_.index))

va_df = va_.to_frame().reset_index()
dict_prob = va_df.set_index(va_df[f]).to_dict()[0]

ideal_loss = log_loss(va_y, np.array([dict_prob[c[0]] for c in va_x]).reshape(-1,1))
model_loss = log_loss(va_y, model.predict(va_x ))
model_loss_ = log_loss(va_y, model_.predict(va_x))
print('ideal-model', 'ideal-model_', 'model_-model')
print('%.8f, %.8f, %.8f'%(ideal_loss-model_loss , model_loss_-model_loss,ideal_loss-model_loss_))

print(va_-np.ravel(model_.predict(va_.index)), '\nBalanced model\n',va_-np.ravel(model.predict(va_.index)) )


model.save('balanced_tl_%s_%.6f.h5'%(f,ideal_loss-model_loss))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 1, 64)             32256     
_________________________________________________________________
flatten_2 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 36,481
Trainable params: 36,481
Non-trainable params: 0
_________________________________________________________________
Train on 47645902 samples, validate on 374953 samples
Epoch 1/2
47645902/47645902 [==============================] - 59s - loss: 0.2011 - 

In [ ]:
va_df = va_.to_frame().reset_index()
dict_prob = va_df.set_index(va_df[f]).to_dict()[0]

ideal_loss = log_loss(va_y, np.array([dict_prob[c[0]] for c in va_x]).reshape(-1,1))
model_loss = log_loss(va_y, model.predict(va_x ))
model_loss_ = log_loss(va_y, model_.predict(va_x))
print('ideal-model', 'ideal-model_', 'model_-model')
print('%.8f, %.8f, %.8f'%(ideal_loss-model_loss , model_loss_-model_loss,ideal_loss-model_loss_))

print(va_-np.ravel(model_.predict(va_.index)), '\nBalanced model\n',va_-np.ravel(model.predict(va_.index)) )


model.save('balanced_tl_%s_%.6f.h5'%(f,ideal_loss-model_loss))
